In [1]:
import crispr as cr
from crispr.class_sc import Omics
from crispr.processing.guide_rna import detect_guide_targets    
import re
import copy
import scanpy as sc
import copy
import os
import seaborn as sns
import matplotlib.pyplot as plt
import functools
import pandas as pd
import numpy as np

file_path = dict(directory=crd, subdirectory_mtx=subd, file_protospacer=proto) 
kws_init = dict(assay=None, assay_protein=None, col_sample_id=None,
                col_gene_symbols="gene_symbols",
                col_cell_type="predicted_labels",
                col_perturbed="perturbation",
                col_guide_rna="feature_call",
                col_num_umis="num_umis",
                col_condition="target_gene_name",
                key_control="NT", key_treatment="KD")

self = cr.Crispr(file_path, **kws_init, kws_process_guide_rna=None)  # no processing of gRNA yet

col_guide_rna = "feature_call"
col_num_umis = "num_umis"
guide_split = "-"
key_control_patterns = ["CTRL"]
key_control = "NT"
feature_split = "|"
min_n_target_control_drop = 100
max_pct_control_drop = 75
min_pct_dominant = 80
min_pct_avg_n = 40
remove_multi_transfected = True
col_guide_rna_new = self._columns["col_target_genes"]
conserve_memory = False

kws_process_guide_rna = dict(
    col_guide_rna = "feature_call",
    col_num_umis = "num_umis",
    guide_split = "-",
    key_control_patterns = ["CTRL"],
    key_control = "NT",
    feature_split = "|",
    min_n_target_control_drop = 100,
    max_pct_control_drop = 75,
    min_pct_dominant = 80,
    min_pct_avg_n = 40,
    remove_multi_transfected = True,
    col_guide_rna_new = self._columns["col_target_genes"],
    conserve_memory = False)

ann = self.adata.copy()
ann.obs = ann.obs.iloc[:, :3]
ann.obs.columns = [re.sub("_original", "", x) for x in ann.obs.columns]

# Then copy-paste the code from the crispr/processing/guide_rna.py functions to run line-by-line